In [1]:
import re
import dill
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# Scrape race pages

In [17]:
race_ids = list(range(1, 13000))
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/race/{i}') for i in race_ids]

In [18]:
for i, future in enumerate(futures):
    dill.dump(future.result().text, open(f'C:/data/results/{i}.pkd', 'wb'))

## Extract race metadata

In [20]:
def get_data(text):
    name = ''
    date = ''
    loc = ''
    json_url = ''
    weather = ''
    strava_url = ''
    coord = None
    
    if 'No data' not in text:
        soup = BeautifulSoup(text)
        resultstitle = soup.select('div#resultstitle')[0].text
        split = resultstitle.split('•')

        regex = re.compile(r'([A-Za-z]{3})\s+(\d{1,2})\s+(\d{4})')
        name = split[0].strip()
        month, day, year = regex.findall(split[1].strip())[0]
        date = datetime.strptime(' '.join([month, day, year]), '%b %d %Y')
        loc = ''
        if len(split) > 2:
            loc = re.split('[\t\n\r]', split[2])[0].strip()
            
        json_url = soup.select('span.downloadoptions')[0].find_all('a')[1]['href']
        weather = soup.select('div#conditionssummary')[0].text.strip()
        strava_url = soup.select('div#stravasegment')[0].select('a')[1]['href']
        
        try:
            coord = re.findall(r'GetMap\("([-\d\.]+):([-\d\.]+)"', text)[0]
        except IndexError:
            pass
            
    return name, date, loc, json_url, weather, strava_url, coord

In [21]:
columns=['name', 'date', 'loc', 'json_url', 'weather', 'strava_url', 'coord']
df = pd.DataFrame(columns=columns)
for i in race_ids:  # loop takes ~2 hour or so
    if i % 100 == 0:
        print(i)
    if i in [12533, 12534]:  # these broke for some reason >:|
        continue
    text = dill.load(open(f'C:/data/results/{i}.pkd', 'rb'))
    data = get_data(text)
    df.loc[i] = {c: d for c, d in zip(columns, data)}

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900


FileNotFoundError: [Errno 2] No such file or directory: 'C:/data/results/12999.pkd'

In [22]:
df.to_pickle('C:/data/results/df.pkl')

In [60]:
df = pd.read_pickle('C:/data/results/df.pkl')
df = df[df['json_url'] != '']
df['weather'] = df['weather'].str.split('\n').str.get(-1)  # sometimes the weather column also got "Series: " information

In [88]:
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
12158,Herbalife Team MS Criterium Series,2017-08-01 00:00:00,"Kernersville, NC",downloadrace.php?raceID=12159&json=1,"Partly Cloudy, 82 degrees, wind 1 mph",http://app.strava.com/segments/,"(36.1195182800293, -80.072509765625)"
5786,UVM Green Mountain Weekend RR,2015-04-11 00:00:00,"Charlotte, VT",downloadrace.php?raceID=5787&json=1,"Mostly Cloudy, 44 degrees, wind 12 mph",http://app.strava.com/segments/,None
4481,Richmond Road Race,2014-05-18 00:00:00,"Beaverdam, VA",downloadrace.php?raceID=4482&json=1,"Clear, 63 degrees, wind 7 mph",http://app.strava.com/segments/,None
4508,Berkshire Cycling Classic,2014-06-14 00:00:00,"Lenox, MA",downloadrace.php?raceID=4509&json=1,"Clear, 63 degrees, wind 13 mph",http://app.strava.com/segments/,None
8217,Mt Whitney Stage Race - Saturday,2016-08-27 00:00:00,"Lone Pine, CA",downloadrace.php?raceID=8218&json=1,"Clear, 83 degrees, wind 7 mph",http://app.strava.com/segments/,None


In [40]:
df.loc[3635]['weather']

'Series: \nBUMPS 2013\n\n\n\n\n\n\r\n       \r\n        Clear, 55  degrees, wind 7 mph'

## Download JSON results

In [4]:
urls = df['json_url']
urls.head()

2    downloadrace.php?raceID=12536&json=1
3    downloadrace.php?raceID=12537&json=1
4    downloadrace.php?raceID=12538&json=1
5    downloadrace.php?raceID=12539&json=1
6    downloadrace.php?raceID=12540&json=1
Name: json_url, dtype: object

In [9]:
for i, url in urls[:10].iteritems():
    print(i, url)
    full_url = f'https://results.bikereg.com/{url}'
    df_race = pd.read_json(full_url)
    columns = [str(i) for i in range(28)]  # Each column repeated as an "index" for some reason
    df_race = df_race.drop(columns=columns)

2 downloadrace.php?raceID=12536&json=1
3 downloadrace.php?raceID=12537&json=1
4 downloadrace.php?raceID=12538&json=1
5 downloadrace.php?raceID=12539&json=1
6 downloadrace.php?raceID=12540&json=1
7 downloadrace.php?raceID=12541&json=1
9 downloadrace.php?raceID=12543&json=1
10 downloadrace.php?raceID=12544&json=1
11 downloadrace.php?raceID=12545&json=1
12 downloadrace.php?raceID=12546&json=1


In [7]:
df_race.topickle('C:/data/results/races/')

'downloadrace.php?raceID=12546&json=1'